# 攻防比と攻防関数

## 1. 攻防比

攻防比とは、物理攻撃における攻撃側の攻撃力と被ダメージ側の防御力の比のことである。
遠隔攻撃の場合は、攻撃側の飛攻が参照される。

## 2. 攻防関数

攻防関数とは、攻防比を引数に取る関数のことである。
この攻防関数の返り値は、物理ダメージ計算値において乗算される。(参考: [ウェポンスキル](https://wiki.ffo.jp/html/307.html))

### 2.1. 攻防関数の最小値と最大値

攻防関数の分布に関する情報はないが、[最大値と最小値に関する情報は存在する。](https://ffxilogdialy.hatenablog.com/entry/2020/10/11/113000)
攻防比を $N$、攻防関数を $Y(N)$とおくと、攻防関数の最大値 $Y_{max}(N)$と攻防関数の最小値 $Y_{min}(N)$はそれぞれ下記のようになる。

#### 2.1.1. 近接攻撃

$$
Y_{max}(N)=
\begin{cases}
  1.141*N + 0.430 & \text{($N<0.5$)}\\
  1 & \text{($0.5≤N<0.75$)}\\
  1.141*N + 0.145 & \text{($0.75≤N<1.625$)}\\
  N + 0.375 & \text{($1.625≤N$)}
\end{cases}
$$

$$
Y_{min}(N)=
\begin{cases}
  1.141*N - 0.426 & \text{($N<1.25$)}\\
  1 & \text{($1.25≤N<1.5$)}\\
  1.141*N + 0.145 & \text{($1.5≤N<2.375$)}\\
  N - 0.375 & \text{($2.375≤N$)}
\end{cases}
$$

#### 2.1.2. 遠隔攻撃

$$
Y_{max}(N)=
\begin{cases}
  1.141*N + 0.05867 & \text{($N<0.825$)}\\
  1 & \text{($0.825≤N<1.075$)}\\
  1.141*N - 0.226575 & \text{($1.075≤N<1.95142$)}\\
  N + 0.0625 & \text{($1.95142≤N$)}
\end{cases}
$$

$$
Y_{min}(N)=
\begin{cases}
  1.141*N - 0.08395 & \text{($N<0.95$)}\\
  1 & \text{($0.95≤N<1.20$)}\\
  1.141*N - 0.3692 & \text{($1.20≤N<2.07642$)}\\
  N - 0.0625 & \text{($2.07642≤N$)}
\end{cases}
$$


### 2.2. 攻防関数の上限値

攻防関数には上限値が設定されている。

攻防関数の基本値を $Y_{u_{b}}$、 [ダメージ上限アップ](https://wiki.ffo.jp/html/37531.html)値を $A$、 [物理ダメージ上限](https://wiki.ffo.jp/html/37520.html)の%値を $B$ とおくと、 攻防関数の上限値 $Y_{u}(Y_{u_{b}}, A, B)$は下記のように求められる。

$$
Y_{u}(Y_{u_{b}}, A, B)=(Y_{u_{b}} + A) * (1+(B/100))
$$

ここで、 $Y_{u_{b}}$は武器種ごとに下記の値に設定されている。

| 武器種                      | Y_u_b | 攻防比(Y_min=Y_u_b) | 攻防比(Y_max=Y_u_b) |
| --------------------------- | ----- | ------------------- | ------------------- |
| 遠隔武器(射撃以外)          |  3.25 |              3.3125 |              3.1875 |
| 片手武器                    |  3.25 |              3.625  |              2.875  |
| 射撃                        |  3.5  |              3.5625 |              3.4375 |
| 格闘, 両手刀                |  3.5  |              3.875  |              3.125  |
| 両手武器(両手鎌,両手刀以外) |  3.75 |              4.125  |              3.375  |
| 両手鎌                      |  4    |              4.375  |              3.625  |


### 2.3. 攻防関数の上限に達するために必要な攻防比

攻防関数の上限に達するために必要な攻防比を $N_{c}$ と置くと、下記の等式が成り立つ。

$$
Y(N_c) = Y_{u}
$$

この等式より、攻防関数の上限値を引数とした、攻防関数の上限に達するために必要な攻防比の関数 $N_c(Y_u)$ を導出することができる。
攻防関数の最大値が上限に達するために必要な攻防比を $N_{c_{max}}(Y_u)$, 攻防関数の最小値が上限に達するために必要な攻防比を $N_{c_{min}}(Y_u)$ とおくと、それぞれ下記で求めることができる。

#### 2.3.1. 近接攻撃

$$
N_{c_{max}}(Y_{u})= Y_u - 0.375
$$

$$
N_{c_{min}}(Y_{u})= Y_u + 0.375
$$

#### 2.3.2. 遠隔攻撃

$$
N_{c_{max}}(Y_{u})= Y_u - 0.0625
$$

$$
N_{c_{max}}(Y_{u})= Y_u + 0.0625
$$

## 3. 攻防比と攻防関数の関係のグラフの描画

In [ ]:
%pip install ipywidgets nbformat numpy plotly

In [2]:
from enum import Enum
import ipywidgets as widgets
import numpy as np
import plotly.graph_objects as go

class WeaponKind(Enum):
    HAND_TO_HAND = '格闘'
    ONE_HAND = '片手武器'
    GREAT_KAKANA = '両手刀'
    SCYCLE = '両手鎌'
    TWO_HANDS = '両手武器(両手鎌,両手刀以外)'
    MARKSMANSHIP = '射撃'
    RANGED = '遠隔武器(射撃以外)'


def atk_def_func_min_ignore_upper(atk_def_ratio: float, weapon_kind: WeaponKind) -> float:
    def atk_def_func_melee_min(atk_def_ratio: float) -> float:
        if atk_def_ratio < 1.25:
            min = 1.141*atk_def_ratio - 0.426
        elif atk_def_ratio < 1.5:
            min = 1
        elif atk_def_ratio < 2.375:
            min = 1.141*atk_def_ratio - 0.711
        else:
            min = atk_def_ratio - 0.375
        return min    

    def atk_def_func_range_min(atk_def_ratio: float) -> float:
        if atk_def_ratio < 0.95:
            min = 1.141*atk_def_ratio - 0.08395
        elif atk_def_ratio < 1.2:
            min = 1
        elif atk_def_ratio < 2.07642:
            min = 1.141*atk_def_ratio - 0.3692
        else:
            min = atk_def_ratio - 0.0625
        return min

    if weapon_kind in {WeaponKind.MARKSMANSHIP, WeaponKind.RANGED}:
        return atk_def_func_range_min(atk_def_ratio)
    
    return atk_def_func_melee_min(atk_def_ratio)


def atk_def_func_max_ignore_upper(atk_def_ratio: float, weapon_kind: WeaponKind) -> float:
    def atk_def_func_melee_max(atk_def_ratio: float) -> float:
        if atk_def_ratio < 0.5:
            max = 1.141*atk_def_ratio + 0.430
        elif atk_def_ratio < 0.75:
            max = 1
        elif atk_def_ratio < 1.625:
            max = 1.141*atk_def_ratio + 0.145
        else:
            max = atk_def_ratio + 0.375
        return max

    def atk_def_func_range_max(atk_def_ratio: float) -> float:
        if atk_def_ratio < 0.825:
            max = 1.141*atk_def_ratio + 0.05867
        elif atk_def_ratio < 1.075:
            max = 1
        elif atk_def_ratio < 1.95142:
            max = 1.141*atk_def_ratio - 0.226575
        else:
            max = atk_def_ratio + 0.0625
        return max

    if weapon_kind in {WeaponKind.MARKSMANSHIP, WeaponKind.RANGED}:
        return atk_def_func_range_max(atk_def_ratio)
    
    return atk_def_func_melee_max(atk_def_ratio)


def atk_def_func_upper(weapon_kind: WeaponKind, a: float, b: float) -> float:
    if weapon_kind in {WeaponKind.ONE_HAND, WeaponKind.RANGED}:
        yub = 3.25
    elif weapon_kind in {WeaponKind.MARKSMANSHIP, WeaponKind.HAND_TO_HAND, WeaponKind.GREAT_KAKANA}:
        yub = 3.5
    elif weapon_kind in {WeaponKind.TWO_HANDS}:
       yub = 3.75
    elif weapon_kind in {WeaponKind.SCYCLE}:
        yub = 4
    else:
        raise ValueError(f"Invalid weapon_kind: {weapon_kind}")

    return (yub + a) * (1 + b/100)


def atk_def_func_min(atk_def_ratio: float, weapon_kind: WeaponKind, a: float, b: int) -> float:
    return min(atk_def_func_min_ignore_upper(atk_def_ratio, weapon_kind), atk_def_func_upper(weapon_kind, a, b))

def atk_def_func_max(atk_def_ratio: float, weapon_kind: WeaponKind, a: float, b: int) -> float:
    return min(atk_def_func_max_ignore_upper(atk_def_ratio, weapon_kind), atk_def_func_upper(weapon_kind, a, b))


def atk_def_ratio_cap_min(weapon_kind: WeaponKind, a: float, b:int) -> float:
    upper = atk_def_func_upper(weapon_kind, a, b);

    if weapon_kind in {WeaponKind.MARKSMANSHIP, WeaponKind.RANGED}:
        return upper + 0.0625
    
    return upper + 0.375

def atk_def_ratio_cap_max(weapon_kind: WeaponKind, a: float, b:float) -> float:
    upper = atk_def_func_upper(weapon_kind, a, b);

    if weapon_kind in {WeaponKind.MARKSMANSHIP, WeaponKind.RANGED}:
        return upper - 0.0625
    
    return upper - 0.375


def print_atk_def_ratio_cap(weapon_kind: WeaponKind, a: float, b:float):
    cap_min = atk_def_ratio_cap_min(weapon_kind, a, b)
    cap_max = atk_def_ratio_cap_max(weapon_kind, a, b)

    print(f"Ncmin: {cap_min}")
    print(f"Ncmax: {cap_max}")


def display_atk_def_func_graph_and_cap(weapon_kind, a, b):
    print_atk_def_ratio_cap(weapon_kind, a, b)

    xmin = 0
    xmax = 7
    num = 100
    x = np.linspace(xmin, xmax, num)
    y_min = np.vectorize(atk_def_func_min)(x, weapon_kind, a, b)
    y_max = np.vectorize(atk_def_func_max)(x, weapon_kind, a, b)       

    fig = go.FigureWidget()
    fig.add_trace(go.Scatter(x=x, y=y_min, mode='lines', name='min'))
    fig.add_trace(go.Scatter(x=x, y=y_max, mode='lines', name='max'))
    fig.update_layout(
        title='攻防比と攻防関数', 
        xaxis={'title': '攻防比'},
        yaxis={'title': '攻防関数'},
        height=800)

    return fig


def display_widget():
    weapon_list = widgets.Dropdown(
        options=[(weapon_kind.value, weapon_kind) for weapon_kind in WeaponKind],
        description="武器種",
    )

    a = widgets.FloatSlider(
        value=0.0,
        min=0,
        max=1,
        step=0.1,
        description='ダメージ上限アップ:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f'
    )

    b = widgets.FloatSlider(
        value=0,
        min=0,
        max=100,
        step=0.1,
        description='物理ダメージ上限+(%):',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f'
    )
    
    widgets.interact(display_atk_def_func_graph_and_cap, weapon_kind=weapon_list, a=a, b=b)


display_widget()

interactive(children=(Dropdown(description='武器種', options=(('格闘', <WeaponKind.HAND_TO_HAND: '格闘'>), ('片手武器', <…

## 4. 参考文献

* [FF11のログから日記 - 攻防関数の推測と仕様変化のまとめ](https://ffxilogdialy.hatenablog.com/entry/2020/10/11/113000)